# AI Agent using GPT-4o

In [1]:
import requests
import re
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
client = OpenAI()

# Tools

In [3]:
def search_product(product_query):
    url = "https://real-time-amazon-data.p.rapidapi.com/search"

    querystring = {"query":product_query,
                   "page":"1",
                   "country":"US",
                   "sort_by":"RELEVANCE",
                   "product_condition":"ALL",
                   "is_prime":"false",
                   "deals_and_discounts":"NONE"}
    
    headers = {
    	"x-rapidapi-key": "beb3b7a15dmsh8e70c17e4fbf3f8p170668jsnc6a0049e6063",
    	"x-rapidapi-host": "real-time-amazon-data.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    
    return response.json()["data"]["products"][:3]

cart = []
def add_to_cart(product_id, quantity):
    global cart
    cart.append({"product_id": product_id, "quantity": quantity})
    return True

def summarize_cart():
    global cart
    return cart

def buy_product(payment_method):
    global cart
    if payment_method == "COD":
        response = "Order confirmation sent to registered email"
    else:
        response = generate_payment_link(payment_method, cart)
    return response

# Util function
def generate_payment_link(payment_method, cart):
    # TODO: actually implement this function
    return "sample.link.com"

# ReAct Prompt

In [4]:
react_prompt = """You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer.

Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
```
search_product:
usage - search_product: <product_query: str>
e.g. search_product: Samsung Galaxy
Returns a list of products matching the given search query.

add_to_cart:
usage - add_to_cart: <product_id: str>, <quantity: int>
e.g. add_to_cart: 1234, 2
Adds the specified quantity of a product to the shopping cart. Returns confirmation if the product was added successfully.

summarize_cart:
usage - summarize_cart: null
Returns the current items in the shopping cart including product IDs and quantities.

buy_product:
usage - buy_product: <payment_method: Enum["COD", "CreditCard", "DebitCard", "UPI", "NetBanking"]>
e.g. buy_product: COD
Attempts to purchase all items in the cart with the specified payment method. Returns an order confirmation message or a payment link if the method is not COD.
```

Example session:
```
Question: I want to buy a Samsung Galaxy phone. Can you help?
Thought: I should search for Samsung Galaxy phones.
Action: search_product: Samsung Galaxy

PAUSE

Observation: The search returned products:
- SAMSUNG Galaxy A15 5G A Series Cell Phone, 128GB Unlocked Android Smartphone, AMOLED Display, Expandable Storage, Knox Security, Super Fast Charging, Hi-Res Camera, US Version, 2024, Blue Black, Price: $199.99

Answer: The search results for Samsung Galaxy phones have been displayed. If you want to add a product to the cart, please specify the product and the desired quantity.

Question: Please add the SAMSUNG Galaxy A15 5G to the cart.
Thought: The user requested to add the product to the cart but did not specify the quantity. I should clarify the quantity.
Answer: How many units of the SAMSUNG Galaxy A15 5G would you like to add to the cart?

Question: Add 2 units.
Thought: The user specified the quantity. I will now add the product to the cart.
Action: add_to_cart: 1234, 2

PAUSE

Observation: Product added to the cart successfully.

Answer: The SAMSUNG Galaxy A15 5G has been added to the cart with quantity 2.
```
"""

# Execution of ReAct Loop

In [5]:
known_actions = {
    "search_product": search_product,
    "add_to_cart": add_to_cart,
    "summarize_cart": summarize_cart,
    "buy_product": buy_product,  
}

In [6]:
action_re = re.compile('^Action: (\w+): (.*)$')

In [7]:
def react_loop(question_messages, max_turns=5):
    react_loop_history = []
    i = 0
    react_loop_messages = [{"role": "system", "content": react_prompt}] + question_messages
    
    while i < max_turns:
        i += 1
        result = client.chat.completions.create(
                        model="gpt-4o", 
                        messages=react_loop_messages)
        result = result.choices[0].message.content
        react_loop_messages.append({"role": "assistant", "content": result})
        react_loop_history.append({"role": "assistant", "content": result})
        
        print("Result of calling the ReAct prompt loop: ", result)
        
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            
            print("Calling tool: {} with input: {}".format(action, action_input))
            action_input_list = [input.strip() for input in action_input.split(",") if input.strip() != "null"]
            observation = known_actions[action](*action_input_list)
            
            print("Observation:", observation)
            observation_prompt = "Observation: {}".format(observation)
            react_loop_messages.append({"role": "user", "content": observation_prompt})
            react_loop_history.append({"role": "user", "content": observation_prompt})
        else:
            return result, react_loop_history

# Chat without memory

In [8]:
def chat(user_query):
    user_query_object = [{"role": "user", "content": user_query}]
    print("======= Starting ReAct Loop =======")
    response, react_loop_history = react_loop(user_query_object)
    print("======= Ending ReAct Loop =======\n\n")
    response = response.split("Answer:")[-1].strip()
    return response

In [9]:
result = chat("i want to buy pair of pants")
print(result)

======= Starting ReAct Loop =======
Result of calling the ReAct prompt loop:  Thought: I should search for pants to find available options that the user might be interested in purchasing.
Action: search_product: pants
Calling tool: search_product with input: pants
Observation: [{'asin': 'B0BTRMKQW5', 'product_title': 'Wrangler Authentics Men&#x27;s Relaxed Fit Stretch Cargo Pant', 'product_price': '$25.24', 'product_original_price': None, 'currency': 'USD', 'product_star_rating': '4.5', 'product_num_ratings': 22495, 'product_url': 'https://www.amazon.com/dp/B0BTRMKQW5', 'product_photo': 'https://m.media-amazon.com/images/I/61HP4BFFzFL._AC_UL960_FMwebp_QL65_.jpg', 'product_num_offers': 1, 'product_minimum_offer_price': '$25.24', 'is_best_seller': True, 'is_amazon_choice': False, 'is_prime': True, 'climate_pledge_friendly': False, 'sales_volume': '500+ bought in past month', 'delivery': 'FREE delivery Wed, Jan 15 on $35 of items shipped by Amazon', 'has_variations': True, 'product_badge'

In [10]:
result = chat("buy the second one")
print(result)

======= Starting ReAct Loop =======
Result of calling the ReAct prompt loop:  Thought: The user wants to buy the second item in the cart. First, I need to check the contents of the cart to identify the second item.
Action: summarize_cart: null
Calling tool: summarize_cart with input: null
Observation: []
Result of calling the ReAct prompt loop:  Thought: The cart is currently empty, so there is no "second item" to purchase. I should inform the user that the cart is empty and assist them in finding and adding a product to the cart if needed. 

Answer: Your shopping cart is currently empty. If you would like to purchase something, please let me know which product you're interested in, and I can help you add it to your cart.
======= Ending ReAct Loop =======


Your shopping cart is currently empty. If you would like to purchase something, please let me know which product you're interested in, and I can help you add it to your cart.


# Chat with memory

In [11]:
messages = []

def chat(user_query):
    global messages
    messages.append({"role": "user", "content": user_query})
    
    print("======= Starting ReAct Loop =======")
    response, react_loop_history = react_loop(messages)
    print("======= Ending ReAct Loop =======\n\n")
    
    messages += react_loop_history
    
    response = response.split("Answer:")[-1].strip()
    return response

In [12]:
result = chat("i want to buy a shirt, maybe blue in colour")
print(result)

======= Starting ReAct Loop =======
Result of calling the ReAct prompt loop:  Thought: I need to search for blue shirts to help the user find a product they might like to purchase.
Action: search_product: blue shirt
Calling tool: search_product with input: blue shirt
Observation: [{'asin': 'B00NOY3AEW', 'product_title': 'Hanes Men&#x27;s Beefy-T T-Shirt, Heavyweight Cotton Tee, 1 Or 2 Pack, Big &amp; Tall Sizes Available', 'product_price': '$8.56', 'product_original_price': '$14.00', 'currency': 'USD', 'product_star_rating': '4.5', 'product_num_ratings': 59046, 'product_url': 'https://www.amazon.com/dp/B00NOY3AEW', 'product_photo': 'https://m.media-amazon.com/images/I/71jWbUuXHbL._AC_UL960_FMwebp_QL65_.jpg', 'product_num_offers': 1, 'product_minimum_offer_price': '$8.56', 'is_best_seller': False, 'is_amazon_choice': False, 'is_prime': True, 'climate_pledge_friendly': True, 'sales_volume': '500+ bought in past month', 'delivery': 'FREE delivery Wed, Jan 15 on $35 of items shipped by Ama

In [13]:
messages

[{'role': 'user', 'content': 'i want to buy a shirt, maybe blue in colour'},
 {'role': 'assistant',
  'content': 'Thought: I need to search for blue shirts to help the user find a product they might like to purchase.\nAction: search_product: blue shirt'},
 {'role': 'user',
  'content': "Observation: [{'asin': 'B00NOY3AEW', 'product_title': 'Hanes Men&#x27;s Beefy-T T-Shirt, Heavyweight Cotton Tee, 1 Or 2 Pack, Big &amp; Tall Sizes Available', 'product_price': '$8.56', 'product_original_price': '$14.00', 'currency': 'USD', 'product_star_rating': '4.5', 'product_num_ratings': 59046, 'product_url': 'https://www.amazon.com/dp/B00NOY3AEW', 'product_photo': 'https://m.media-amazon.com/images/I/71jWbUuXHbL._AC_UL960_FMwebp_QL65_.jpg', 'product_num_offers': 1, 'product_minimum_offer_price': '$8.56', 'is_best_seller': False, 'is_amazon_choice': False, 'is_prime': True, 'climate_pledge_friendly': True, 'sales_volume': '500+ bought in past month', 'delivery': 'FREE delivery Wed, Jan 15 on $35 of 

In [14]:
result = chat("lets add third one")
print(result)

======= Starting ReAct Loop =======
Result of calling the ReAct prompt loop:  Thought: The user wants to add the third product (Gildan Unisex Adult Dryblend T-Shirt) to the cart, but they have not specified the quantity. I should ask for the quantity.
Answer: How many units of the Gildan Unisex Adult Dryblend T-Shirt would you like to add to your cart?
======= Ending ReAct Loop =======


How many units of the Gildan Unisex Adult Dryblend T-Shirt would you like to add to your cart?


In [15]:
result = chat("2")
print(result)

======= Starting ReAct Loop =======
Result of calling the ReAct prompt loop:  Action: add_to_cart: B07MGYCGY1, 2
Calling tool: add_to_cart with input: B07MGYCGY1, 2
Observation: True
Result of calling the ReAct prompt loop:  Answer: The Gildan Unisex Adult Dryblend T-Shirt has been successfully added to your cart with a quantity of 2. Would you like to proceed with the purchase or need anything else?
======= Ending ReAct Loop =======


The Gildan Unisex Adult Dryblend T-Shirt has been successfully added to your cart with a quantity of 2. Would you like to proceed with the purchase or need anything else?


In [17]:
messages

[{'role': 'user', 'content': 'i want to buy a shirt, maybe blue in colour'},
 {'role': 'assistant',
  'content': 'Thought: I need to search for blue shirts to help the user find a product they might like to purchase.\nAction: search_product: blue shirt'},
 {'role': 'user',
  'content': "Observation: [{'asin': 'B00NOY3AEW', 'product_title': 'Hanes Men&#x27;s Beefy-T T-Shirt, Heavyweight Cotton Tee, 1 Or 2 Pack, Big &amp; Tall Sizes Available', 'product_price': '$8.56', 'product_original_price': '$14.00', 'currency': 'USD', 'product_star_rating': '4.5', 'product_num_ratings': 59046, 'product_url': 'https://www.amazon.com/dp/B00NOY3AEW', 'product_photo': 'https://m.media-amazon.com/images/I/71jWbUuXHbL._AC_UL960_FMwebp_QL65_.jpg', 'product_num_offers': 1, 'product_minimum_offer_price': '$8.56', 'is_best_seller': False, 'is_amazon_choice': False, 'is_prime': True, 'climate_pledge_friendly': True, 'sales_volume': '500+ bought in past month', 'delivery': 'FREE delivery Wed, Jan 15 on $35 of 

In [18]:
cart

[{'product_id': 'B07MGYCGY1', 'quantity': '2'}]

In [19]:
result = chat("actually lets add first one as well with 1 quantity")
print(result)

======= Starting ReAct Loop =======
Result of calling the ReAct prompt loop:  Action: add_to_cart: B00NOY3AEW, 1
Calling tool: add_to_cart with input: B00NOY3AEW, 1
Observation: True
Result of calling the ReAct prompt loop:  Answer: The Hanes Men's Beefy-T T-Shirt has been successfully added to your cart with a quantity of 1. Would you like to proceed with the purchase or is there anything else you need?
======= Ending ReAct Loop =======


The Hanes Men's Beefy-T T-Shirt has been successfully added to your cart with a quantity of 1. Would you like to proceed with the purchase or is there anything else you need?


In [20]:
result = chat("what all is there in my cart")
print(result)

======= Starting ReAct Loop =======
Result of calling the ReAct prompt loop:  Action: summarize_cart: null
Calling tool: summarize_cart with input: null
Observation: [{'product_id': 'B07MGYCGY1', 'quantity': '2'}, {'product_id': 'B00NOY3AEW', 'quantity': '1'}]
Result of calling the ReAct prompt loop:  Answer: Your cart currently contains the following items:

1. **Gildan Unisex Adult Dryblend T-Shirt**
   - Quantity: 2

2. **Hanes Men's Beefy-T T-Shirt**
   - Quantity: 1

If you would like to proceed with the purchase or make any changes, please let me know!
======= Ending ReAct Loop =======


Your cart currently contains the following items:

1. **Gildan Unisex Adult Dryblend T-Shirt**
   - Quantity: 2

2. **Hanes Men's Beefy-T T-Shirt**
   - Quantity: 1

If you would like to proceed with the purchase or make any changes, please let me know!


In [21]:
result = chat("lets checkout")
print(result)

======= Starting ReAct Loop =======
Result of calling the ReAct prompt loop:  Thought: The user wants to proceed with purchasing the items in the cart. I need to confirm the payment method they would like to use.
Answer: Which payment method would you like to use for the purchase? You can choose from the following options: Cash on Delivery (COD), Credit Card, Debit Card, UPI, or Net Banking.
======= Ending ReAct Loop =======


Which payment method would you like to use for the purchase? You can choose from the following options: Cash on Delivery (COD), Credit Card, Debit Card, UPI, or Net Banking.


In [22]:
result = chat("net bank")
print(result)

======= Starting ReAct Loop =======
Result of calling the ReAct prompt loop:  Action: buy_product: NetBanking
Calling tool: buy_product with input: NetBanking
Observation: sample.link.com
Result of calling the ReAct prompt loop:  Answer: Your purchase has been initiated successfully. Please complete the payment using the following link: [Payment Link](sample.link.com). If you need any further assistance, feel free to ask. Thank you for shopping with us!
======= Ending ReAct Loop =======


Your purchase has been initiated successfully. Please complete the payment using the following link: [Payment Link](sample.link.com). If you need any further assistance, feel free to ask. Thank you for shopping with us!


In [23]:
messages = []

In [24]:
result = chat("buy 2 shirts with cash on delivery")
print(result)

======= Starting ReAct Loop =======
Result of calling the ReAct prompt loop:  Thought: I need to search for shirts first in order to find suitable options for the user to purchase. Then, I'll add the desired number of shirts to the cart and proceed with a cash on delivery purchase. Let's start by searching for shirts. 
Action: search_product: shirt
Calling tool: search_product with input: shirt
Observation: [{'asin': 'B08F2P8CS9', 'product_title': 'Fruit of the Loom Men&#x27;s Eversoft Cotton Stay Tucked Crew T-shirt', 'product_price': '$18.61', 'product_original_price': None, 'currency': 'USD', 'product_star_rating': '4.6', 'product_num_ratings': 130884, 'product_url': 'https://www.amazon.com/dp/B08F2P8CS9', 'product_photo': 'https://m.media-amazon.com/images/I/71GX5QGOKsL._AC_UL960_FMwebp_QL65_.jpg', 'product_num_offers': 1, 'product_minimum_offer_price': '$18.61', 'is_best_seller': True, 'is_amazon_choice': False, 'is_prime': True, 'climate_pledge_friendly': True, 'sales_volume': '6